In [74]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from numpy import mean
from numpy import std

npf_train =pd.read_csv('npf_train (1).csv',index_col='date')
npf_train = npf_train.drop(columns=["partlybad", 'id'])
npf_train["class4"] = npf_train["class4"].astype("category")

class2 = np.array(["event"]*npf_train.shape[0],dtype="object")
class2[npf_train["class4"]=="nonevent"] = "nonevent"
npf_train["class2"] = class2
npf_train["class2"] = npf_train["class2"].astype("category")

for i in range(1,101): 
    npf_train.iloc[:,i] = pd.to_numeric(list(npf_train.iloc[:,i]))

x, x_test, y, y_test = train_test_split(npf_train.iloc[:360, 1:-1],npf_train.iloc[:360, -1], test_size=0.2,train_size=0.8)
    

In [75]:
y_test

date
2003-06-25       event
2007-06-24    nonevent
2000-09-29    nonevent
2008-08-07       event
2004-03-14    nonevent
                ...   
2000-04-08       event
2005-09-24    nonevent
2000-01-20    nonevent
2009-01-03    nonevent
2004-09-22    nonevent
Name: class2, Length: 72, dtype: category
Categories (2, object): ['event', 'nonevent']

In [77]:
# train a basic random forest model
rf = RandomForestClassifier()
train_rf = rf.fit(x,y)
print('params: ', train_rf.get_params())

# receiving the score
score_rf = train_rf.score(x_test, y_test)
print('score: ',score_rf)

params:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
score:  0.7777777777777778


In [79]:
# creating a function outputting the results of the model
def pred_class(train_rf, x_test, y_test): # t_model is the trained model
    df = pd.DataFrame()
    df["x"] = [x_test.iloc[i, :] for i in range(len(x_test))]
    df["y_true"] = np.array(y_test)
    df["y_pred"] = train_rf.predict(x_test)
    proba_event= train_rf.predict_proba(x_test)[:,0]
    df["y_pred_proba"] = proba_event
    df["correct?"] = [df.iloc[i,1] == df.iloc[i,2] for i in range(len(df))]
    
    return df

pred_class(train_rf, x_test, y_test)

,x,y_true,y_pred,y_pred_proba,correct?
0,CO2168.mean 366.435918 CO2168.std 2.5...,event,event,0.74,True
1,CO2168.mean 375.603834 CO2168.std 1.9...,nonevent,event,0.89,False
2,CO2168.mean 374.959348 CO2168.std 2.9...,nonevent,nonevent,0.01,True
3,CO2168.mean 366.638765 CO2168.std 4.9...,event,event,0.89,True
4,CO2168.mean 385.653932 CO2168.std 0.4...,nonevent,nonevent,0.24,True
...,...,...,...,...,...
67,CO2168.mean 373.972781 CO2168.std 1.1...,event,event,1.00,True
68,CO2168.mean 371.770826 CO2168.std 5.0...,nonevent,nonevent,0.02,True
69,CO2168.mean 374.884615 CO2168.std 0.4...,nonevent,nonevent,0.11,True
70,CO2168.mean 396.713000 CO2168.std 0.2...,nonevent,nonevent,0.30,True


In [58]:
params = pd.DataFrame()
params["n-estimator"] = []
params["model"] = []
params["score"] = []

for i in (100,150, 200, 250, 300, 500,1000):
    rf_gini_nestimators = RandomForestClassifier(n_estimators=i, criterion='gini')
    rf_gini_nestimators=rf_gini_nestimators.fit(X, y)
    rf_gini_nestimators_score=rf_gini_nestimators.score(X_test, y_test)
    params=params.append({'n-estimator': i,'model':rf_gini_nestimators, 'score': rf_gini_nestimators_score}, ignore_index=True)
params



,n-estimator,model,score
0,100.0,"(DecisionTreeClassifier(max_features='auto', r...",0.897959
1,150.0,"(DecisionTreeClassifier(max_features='auto', r...",0.897959
2,200.0,"(DecisionTreeClassifier(max_features='auto', r...",0.897959
3,250.0,"(DecisionTreeClassifier(max_features='auto', r...",0.908163
4,300.0,"(DecisionTreeClassifier(max_features='auto', r...",0.908163
5,500.0,"(DecisionTreeClassifier(max_features='auto', r...",0.918367
6,1000.0,"(DecisionTreeClassifier(max_features='auto', r...",0.918367


In [60]:
params_depth = pd.DataFrame()
params_depth["max_depth"] = []
params_depth["model"] = []
params_depth["score"] = []

for i in range(1,12):
    rf_gini_depth = RandomForestClassifier(max_depth=i, criterion='gini', n_estimators=1000)
    rf_gini_depth=rf_gini_depth.fit(X, y)
    rf_gini_depth_score=rf_gini_depth.score(X_test, y_test)
    params_depth=params_depth.append({'max_depth': i,'model':rf_gini_depth, 'score': rf_gini_depth_score}, ignore_index=True)
params_depth

,max_depth,model,score
0,1.0,"(DecisionTreeClassifier(max_depth=1, max_featu...",0.826531
1,2.0,"(DecisionTreeClassifier(max_depth=2, max_featu...",0.897959
2,3.0,"(DecisionTreeClassifier(max_depth=3, max_featu...",0.908163
3,4.0,"(DecisionTreeClassifier(max_depth=4, max_featu...",0.908163
4,5.0,"(DecisionTreeClassifier(max_depth=5, max_featu...",0.908163
5,6.0,"(DecisionTreeClassifier(max_depth=6, max_featu...",0.908163
6,7.0,"(DecisionTreeClassifier(max_depth=7, max_featu...",0.908163
7,8.0,"(DecisionTreeClassifier(max_depth=8, max_featu...",0.908163
8,9.0,"(DecisionTreeClassifier(max_depth=9, max_featu...",0.918367
9,10.0,"(DecisionTreeClassifier(max_depth=10, max_feat...",0.918367


In [62]:
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestClassifier()
parametres={'n_estimators': range(100, 1001),
            'criterion': ['entropy', 'gini'],
            'min_samples_split': range(2,10),
            'max_depth': [None,1,3,5,7,9],
            'max_features': range(1,11),  
            'min_samples_leaf': [1,3,5],
        'bootstrap': ['True', 'False']}
search=RandomizedSearchCV(rf, parametres)
search.fit(X, y)
best_tree=search.best_estimator_
best_tree

RandomForestClassifier(bootstrap='True', max_depth=7, max_features=8,
                       min_samples_leaf=3, min_samples_split=4,
                       n_estimators=712)

In [65]:
X,Y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=3)
# define the model
model = RandomForestClassifier(bootstrap='True', max_depth=9, max_features=8,
                       min_samples_split=4,min_samples_leaf=3, n_estimators=951)
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.890 (0.023)


In [67]:
from sklearn.model_selection import cross_val_score

cv10 = []
cv50 = []

cv = cross_val_score(model, X,Y, cv = 10)
print(cv)
cv = cross_val_score(model, X,Y, cv = 50)
print(cv)
    


[0.9  0.93 0.88 0.9  0.86 0.94 0.86 0.87 0.87 0.91]
[0.9  0.9  0.85 1.   0.9  0.9  0.9  0.95 0.95 0.95 0.85 1.   0.85 0.8
 0.9  0.95 0.85 0.95 0.85 0.95 0.95 0.75 0.85 0.95 0.85 0.95 0.95 0.95
 0.85 1.   0.85 0.85 0.9  0.8  0.9  0.85 0.8  0.9  0.95 0.9  0.9  0.85
 0.85 0.85 0.85 0.9  0.85 0.9  0.9  1.  ]


In [85]:
#for multiclass predictions
npf_train =pd.read_csv('npf_train (1).csv',index_col='date')
npf_train = npf_train.drop(columns=["partlybad", 'id'])
npf_train["class4"] = npf_train["class4"].astype("category")

class2 = np.array(["event"]*npf_train.shape[0],dtype="object")
class2[npf_train["class4"]=="nonevent"] = "nonevent"
npf_train["class2"] = class2
npf_train["class2"] = npf_train["class2"].astype("category")

for i in range(1,101): 
    npf_train.iloc[:,i] = pd.to_numeric(list(npf_train.iloc[:,i]))

x, x_test, y, y_test = train_test_split(npf_train.iloc[:360, 1:-1],npf_train.iloc[:360, 0], test_size=0.2,train_size=0.8)
y

date
2005-08-31          Ib
2008-09-07    nonevent
2005-01-17    nonevent
2000-06-21          II
2000-03-28          II
                ...   
2007-05-06          II
2005-10-19    nonevent
2007-05-02          II
2007-04-18          Ib
2007-06-03          II
Name: class4, Length: 288, dtype: category
Categories (4, object): ['II', 'Ia', 'Ib', 'nonevent']

In [86]:
# train a basic random forest model
rf = RandomForestClassifier()
train_rf = rf.fit(x,y)
print('params: ', train_rf.get_params())

# receiving the score
score_rf = train_rf.score(x_test, y_test)
print('score: ',score_rf)

params:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
score:  0.5694444444444444


In [87]:
X,Y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=3)
# define the model
model = RandomForestClassifier(bootstrap='True', max_depth=9, max_features=8,
                       min_samples_split=4,min_samples_leaf=3, n_estimators=951)
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.891 (0.022)


In [88]:
cv = cross_val_score(model, X,Y, cv = 10)
print(cv)
cv = cross_val_score(model, X,Y, cv = 50)
print(cv)
    


[0.9  0.93 0.87 0.9  0.88 0.94 0.86 0.87 0.87 0.89]
[0.9  0.9  0.8  1.   0.9  0.9  0.95 0.95 0.95 0.95 0.85 1.   0.8  0.8
 0.85 0.95 0.85 0.95 0.85 0.95 0.95 0.8  0.85 0.95 0.9  0.95 0.95 0.95
 0.85 1.   0.85 0.85 0.9  0.85 0.9  0.85 0.75 0.9  0.95 0.9  0.9  0.85
 0.85 0.85 0.85 0.9  0.85 0.9  0.9  1.  ]


# Unsupervised learning